# LAB3,5: Opitcal Flow
### The task is an additional / not obligatory task:
### Please write a program in Python and OpenCV libraries that includes functionality as follow:

1. The software will read the movie file with the name given in the command line parameters
2. It will automatically mark good tracking points on the screen and will follow its throughout the duration of the video
3. It will open a window in which the movie and the path of the selected points will be displayed.

### Please deliver the code along with the video generated from its operation via the Teams
### platform to the next classes.
#### Good luck

Mateusz Andrzejewski

---

### Created by: `Jakub Góralczyk`

---

In [2]:
import numpy as np
import cv2

In [5]:
def optical_flow(video_path:str ="videos\\plane.mov"):
   
    video_input = cv2.VideoCapture(video_path)
    # params for Shi-Tomasi corner detection
    if video_input is None:
        print(f'Unable to open video at {video_input}')

    video_height = int(video_input.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_width  = int(video_input.get(cv2.CAP_PROP_FRAME_WIDTH))
    writer = cv2.VideoWriter('videos\\plane_output.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(video_width,video_height))

    feature_params = dict( maxCorners = 100,
                           qualityLevel = 0.3,
                           minDistance = 7,
                           blockSize = 7 )

    # Parameters for Lucas Kanade optical flow
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 2,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Create some random colors
    color = np.random.randint(0,255,(100,3))

    # Take first frame and find corners in it
    frame_grabbed, old_frame = video_input.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY) # convert to grayscale
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    while(frame_grabbed):
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        frame_grabbed,frame = video_input.read()
        try:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convert to grayscale
        except Exception:
            print()
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) # p0 is the previous frame, p1 is the next frame and st is the status of the points

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]
        
        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new, good_old)):
            a,b = new.ravel() 
            c,d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2) 
            frame = cv2.circle(frame, (int(a), int(b)), 3, color[i].tolist(),-1)
        try:
            img = cv2.add(frame,mask)
        except Exception:
            print()
        cv2.imshow('frame',img) # show frame
        writer.write(img) # write frame to video file
        k = cv2.waitKey(30) & 0xff # wait for 'esc' key press 
        
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2) 

    video_input.release()
    writer.release()
    cv2.destroyAllWindows()

In [6]:
optical_flow()